In [1]:
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box, shape
import rasterio
import os
import matplotlib.pyplot as plt
import pandas as pd
import rasterio.features

In [7]:

region = "Limburg"
Output_dir = Path(rf"P:\bovenregionale-stresstest-hwn\Analysis\{region}\Outputs")
hazard_graph = gpd.read_file(Output_dir.joinpath("base_network_hazard.gpkg"), driver='GPKG')

In [8]:
import geopandas as gpd
import pandas as pd

gdf = hazard_graph.copy()
# Step 1: Initialize the score column
gdf['Flood_uncertainty'] = 0

# Step 2: Add 1 if EV1_fr < 0.2
gdf.loc[gdf['EV1_fr'] < 0.3, 'Flood_uncertainty'] += 1

# Step 3: Add 1 if both EV1_me and EV1_ma <= 0.3
gdf.loc[(gdf['EV1_me'] <= 0.3) & (gdf['EV1_ma'] <= 0.3), 'Flood_uncertainty'] += 1
#gdf.loc[gdf['EV1_me'] < 0.3, 'Flood_uncertainty'] += 1

# Step 4: Define the isolation check
def is_isolated(index, flooded_series):
    if not flooded_series.iloc[index]:
        return False
    neighbors = []
    if index > 0:
        neighbors.append(flooded_series.iloc[index - 1])
    if index < len(flooded_series) - 1:
        neighbors.append(flooded_series.iloc[index + 1])
    return not any(neighbors)

# Step 5: Apply isolation logic and update score
gdf['is_flooded'] = gdf['Flood_uncertainty'] > 0
gdf['is_isolated'] = [is_isolated(i, gdf['is_flooded']) for i in range(len(gdf))]
gdf.loc[gdf['is_isolated'], 'Flood_uncertainty'] += 1


In [9]:
gdf = gdf.set_crs("EPSG:28992", allow_override=True)
gdf.to_file(Output_dir.joinpath("flooded_segments_case_based.gpkg"), driver='GPKG')

In [10]:

ev_cols = [col for col in gdf.columns if col.startswith('EV')]
gdf.loc[gdf['Flood_uncertainty'] > 0, ev_cols] = 0



In [11]:
import geopandas as gpd

# Calculate flooded length per segment
gdf['flooded_length'] = gdf['EV1_fr'] * gdf['length']

# Define aggregation dictionary
ev_cols = [col for col in gdf.columns if col.startswith('EV')]
agg_dict = {
    'length': 'sum',
    'flooded_length': 'sum',
    **{col: ['mean', 'max', 'median'] for col in ev_cols}
}

# Dissolve geometries and aggregate
gdf_dissolved = gdf.dissolve(by='NETWERKSCH', aggfunc=agg_dict, as_index=False)

# Flatten multi-level columns
gdf_dissolved.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
    for col in gdf_dissolved.columns
]

# Rename for clarity
gdf_dissolved = gdf_dissolved.rename(columns={
    'length_sum': 'total_length',
    'flooded_length_sum': 'flooded_length'
})

# Calculate fraction flooded
gdf_dissolved['fraction_flooded'] = gdf_dissolved['flooded_length'] / gdf_dissolved['total_length']

# Convert to GeoDataFrame and set CRS
gdf_agg = gpd.GeoDataFrame(gdf_dissolved, geometry='geometry', crs=gdf.crs)
gdf_agg = gdf_agg.set_crs("EPSG:28992", allow_override=True)

# Save to file
gdf_agg.to_file(Output_dir.joinpath("Aggregated_flooded_segments.gpkg"), driver='GPKG')


CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_contents SET min_x = 163238.9851999989, min_y = 307279.826000001, max_x = 212732.3459999999, max_y = 409891.5467784372 WHERE lower(table_name) = lower('Aggregated_flooded_segments') AND Lower(data_type) = 'features') failed: disk I/O error"

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_contents SET min_x = 163238.9851999989, min_y = 307279.826000001, max_x = 212732.3459999999, max_y = 409891.5467784372 WHERE lower(table_name) = lower('Aggregated_flooded_segments') AND Lower(data_type) = 'features') failed: disk I/O error"
